In [ ]:
import torch
import torch.nn as nn
from torch.functional import F
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
from collections import Counter
import random
from IPython.display import clear_output
import math

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
model = torch.load('saves/model')
model

In [ ]:
import pickle
tok_tfm = pickle.load(open('saves/tok_tfm.p', 'rb'))

In [ ]:
def generate(sm_base, gen_len, vocab_sz, inp):
    with torch.no_grad():
        for _ in range(gen_len):
            batched = inp[None,:] # add batch dimension
            preds = model(batched)
            logits = preds[0,-1,:] # get only the last predicted token
            logits = logits.numpy() # convert to numpy for weighted random choice functionality
            logits = logits[1:] # don't predict xxunk (position 0)
            exped = sm_base**logits # like softmax, but with an adjustable base instead of e
            probs = exped / exped.sum() 
            new = np.random.choice(np.arange(1, vocab_sz), size=1, p=probs) # don't predict xxunk
            new_t = torch.tensor(new)
            inp = torch.cat([inp, new_t])
        return inp

In [ ]:
def make_text(sm_base,gen_len,inp):
    inp = inp.split(' ')
    inp = tok_tfm.encode(inp)
    output = generate(sm_base, gen_len, tok_tfm.count, inp)
    joined = ' '.join(tok_tfm.decode(output))
    fixed = re.sub(r' ([.,?:;’”])', '\\1', joined)
    fixed = re.sub(r'([“‘]) ', '\\1', fixed)
    fixed = re.sub(r'’ s', '’s', fixed)
    return fixed

In [ ]:
# np.random.seed(1)

In [ ]:
make_text(4,100,"the only")